This file forecasts derailment/subreddit using BERT.

# 1. Install requirements

In [49]:
model_name = "distilroberta-base"
test_size = 0.2
labels = [True, False]  # replace with your real labels
num_labels = len(labels)
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 2. Read and Prepare the Dataset

In [50]:
import pandas as pd
import pickle
import os

In [51]:
corpus = os.path.join('..', '..', 'Data', 'processed', 'email_label_dict_balanced.pkl')

In [52]:
# Load the balanced dictionary
with open(corpus, 'rb') as f:
    emails_and_tag = pickle.load(f)

print(f"Loaded {len(emails_and_tag):,} emails")

# Check class distribution
casual_count = sum(1 for v in emails_and_tag.values() if v == 0)
business_count = sum(1 for v in emails_and_tag.values() if v == 1)
print(f"Casual: {casual_count:,} | Business: {business_count:,}")
print(f"Ratio: 1:{business_count/casual_count:.1f}")


Loaded 28,536 emails
Casual: 14,268 | Business: 14,268
Ratio: 1:1.0


In [53]:
import random

emails = []
is_business = []

for key, value in emails_and_tag.items():
    emails.append(key)
    is_business.append(value) 


# Zip the lists together
combined = list(zip(emails, is_business))

# Shuffle the combined list
random.shuffle(combined)

# Unzip back into two lists
email_shuffled, is_buisness_shuffled = zip(*combined)

# Convert back to lists (optional)
email_shuffled = list(email_shuffled)
is_buisness_shuffled = list(is_buisness_shuffled)

pre_dataframe = {"email": email_shuffled, "labels":is_buisness_shuffled}
df = pd.DataFrame(pre_dataframe)

In [54]:
df.head()

,email,labels
0,---------------------- Forwarded by Matthew Le...,1
1,Subject: EBS GA Conference Call\n\nOur weekly ...,1
2,Subject: Re: morning 4/30\n\nI'm glad you had ...,0
3,"Subject: RE: ripple claims\n\nSteve,\n\tWe are...",1
4,Subject: Re: Mary Kay Makeup\n\nI have revised...,0


### Label Encoder

In [55]:
from sklearn import preprocessing

In [56]:
# le = preprocessing.LabelEncoder()
# le.fit(df["derails"].tolist())
# df['label'] = le.transform(df["derails"].tolist())

In [57]:
df.head()

,email,labels
0,---------------------- Forwarded by Matthew Le...,1
1,Subject: EBS GA Conference Call\n\nOur weekly ...,1
2,Subject: Re: morning 4/30\n\nI'm glad you had ...,0
3,"Subject: RE: ripple claims\n\nSteve,\n\tWe are...",1
4,Subject: Re: Mary Kay Makeup\n\nI have revised...,0


### Train/Test/Val Split

In [58]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [59]:
# First split off 20% for test
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the remaining 80% into 60% train and 20% val
# 20% out of the remaining 80% is 0.25 of that chunk
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)

# Hugging Face format
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "validation": Dataset.from_pandas(df_val.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
})


### Convert to correct dataframe

In [60]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]


### Tokenizer

In [61]:
from transformers import AutoTokenizer

In [62]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["email"], truncation=True)

In [63]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 17121/17121 [00:02<00:00, 6165.56 examples/s]


In [64]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 5708/5708 [00:00<00:00, 6518.38 examples/s]


# 3. Initialize Model

In [65]:
from transformers import AutoModelForSequenceClassification

In [66]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4. Train model

In [67]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

In [68]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [69]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"eval_accuracy": acc}

In [70]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    logging_strategy="epoch",
    eval_strategy="epoch",                 # ✅ clean & modern
    save_strategy="epoch",                 # ✅ matches eval
    save_total_limit=2,                    # ✅ keeps best 2 checkpoints
    load_best_model_at_end=True,           # ✅ will restore best version
    metric_for_best_model="eval_accuracy", # ✅ or use "eval_accuracy"
    greater_is_better=True,                # ✅ for accuracy
    report_to="none",
    learning_rate=3e-5,
    per_device_train_batch_size=128,       # for speed
    per_device_eval_batch_size=256,        # speed
    fp16=True,                             # speed
    warmup_steps=500,
    weight_decay=0.05,
)

trainer = Trainer(
    # model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]  # 👈 Add this line
)


/tmp/ipykernel_2346843/3986559634.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
def compute_objective(metrics):
    return metrics["eval_accuracy"]


Now re-generate the model with the best hyperparameters

In [72]:
# 4. Retrain and save
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.585600,0.382653,0.839348
2,0.323400,0.291900,0.882621
3,0.265300,0.277933,0.891381
4,0.225300,0.269872,0.894184
5,0.167900,0.264923,0.904520


TrainOutput(global_step=670, training_loss=0.3135089988139138, metrics={'train_runtime': 116.5126, 'train_samples_per_second': 734.728, 'train_steps_per_second': 5.75, 'total_flos': 1.133987166194688e+16, 'train_loss': 0.3135089988139138, 'epoch': 5.0})

In [73]:
# trainer.save_model('BERT')

# 5. Evaluate Model

In [74]:
from sklearn.metrics import classification_report

In [75]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['labels'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      8558
           1       0.96      0.95      0.96      8563

    accuracy                           0.96     17121
   macro avg       0.96      0.96      0.96     17121
weighted avg       0.96      0.96      0.96     17121



In [76]:
# output = trainer.predict(tokenized_test)
# logits = output.predictions  # raw model output before argmax
# preds = np.argmax(logits, axis=1)  # class with highest score

# GT = df_val['labels'].tolist()  
# print(classification_report(GT, preds))

output = trainer.predict(tokenized_test)

logits = output.predictions
preds = np.argmax(logits, axis=1)

# Use test set labels, not val set
GT = df_test['labels'].tolist()  # <-- Change df_val to df_test

print(classification_report(GT, preds))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      2832
           1       0.91      0.90      0.90      2876

    accuracy                           0.90      5708
   macro avg       0.90      0.90      0.90      5708
weighted avg       0.90      0.90      0.90      5708



In [77]:
trainer.evaluate()

{'eval_accuracy': 0.9045199719691661,
 'eval_loss': 0.2649230360984802,
 'eval_runtime': 2.9268,
 'eval_samples_per_second': 1950.236,
 'eval_steps_per_second': 7.858,
 'epoch': 5.0}

In [78]:
trainer.save_model("./my_email_classifier")
tokenizer.save_pretrained("./my_email_classifier")

('./my_email_classifier/tokenizer_config.json',
 './my_email_classifier/special_tokens_map.json',
 './my_email_classifier/vocab.json',
 './my_email_classifier/merges.txt',
 './my_email_classifier/added_tokens.json',
 './my_email_classifier/tokenizer.json')